## Tutorial Steps:

1. **Download Example ISS Dataset:** Obtain the provided ISS dataset to work with.

2. **Optional: Deconvolution and Maximum Intensity Projection:** You have the option to apply deconvolution and create maximum intensity projections from the raw image data.

3. **Stitching Image Data:** Combine the image data using stitching techniques.

4. **Decode Image Data:** Decode the stitched image data.

5. **Quality Control and Visualization:** Evaluate the results through quality control measures and visualize them.


# Installation

We recommened installing all the necessary packages using [miniconda](https://docs.conda.io/en/latest/miniconda.html)
or [Anaconda](https://www.anaconda.com/products/individual)

Begin by creating a named conda environment with python 3.10:
```bash
conda create -y -n iss_decoding python=3.10
```

Activate the conda environment:
```bash
conda activate iss_decoding
```

In the activated environment, install the following packages:
```bash
conda install -y -c conda-forge numpy scipy matplotlib networkx scikit-image=0.19 scikit-learn "tifffile>=2023.3.15" zarr pyjnius blessed
pip install ashlar pandas tqdm seaborn torch napari[all]
```

We also need to install `libvips` and `pyvips`. On Linux and macOS, this can be done through conda:

```bash
conda install -c conda-forge libvips pyvips
```

on Windows you can download a pre-compiled binary from the libvips website.

https://libvips.github.io/libvips/install.html


OBS! You will also need to add `vips-dev-x.y\bin` to your PATH so that pyvips can find all the DLLs it needs. You can either do this in the Advanced System Settings control panel,

Next, install `pyvips` as

```bash
pip install pyvips
```


### Step 1. Download ISS Data
To begin, download the ISS toy dataset by clicking on the following link: [ISS Toy Dataset](https://drive.google.com/file/d/1zYoUHDOCIuvyJBWj-KQnbVMM4THBf7ll/view?usp=drive_link).


Once the dataset is downloaded, take a moment to examine the file names and familiarize yourself with their naming conventions. The files adhere to the following naming pattern: `stage{stage}_round{round}_z{z}_channel{channel}.tif`, where the placeholders correspond to the numerical identifiers for the stage position, staining round, z level, and channel.


Next, we'll proceed to load the dataset into an `ISSDataContainer` class. This class is designed to facilitate dataset management without the need to load the entire contents into memory simultaneously.

In [2]:
from imaging_utils import ISSDataContainer

# Create the container
issdata = ISSDataContainer()

# Add images
# join('downloads', 'stage{stage}_rounds{round}_z{z}_channel{channel}.tif')
pattern = 'decoding_tutorial\\S{stage}_R{round}_C{channel}_Z{z}.tif'
issdata.add_images_from_filepattern(pattern)

Added decoding_tutorial\S0_R0_C0_Z17.tif. Stage: 0, Round: 0, Channel: 0
Added decoding_tutorial\S0_R0_C1_Z17.tif. Stage: 0, Round: 0, Channel: 1
Added decoding_tutorial\S0_R0_C2_Z17.tif. Stage: 0, Round: 0, Channel: 2
Added decoding_tutorial\S0_R0_C3_Z17.tif. Stage: 0, Round: 0, Channel: 3
Added decoding_tutorial\S0_R0_C4_Z17.tif. Stage: 0, Round: 0, Channel: 4
Added decoding_tutorial\S1_R0_C0_Z17.tif. Stage: 1, Round: 0, Channel: 0
Added decoding_tutorial\S1_R0_C1_Z17.tif. Stage: 1, Round: 0, Channel: 1
Added decoding_tutorial\S1_R0_C2_Z17.tif. Stage: 1, Round: 0, Channel: 2
Added decoding_tutorial\S1_R0_C3_Z17.tif. Stage: 1, Round: 0, Channel: 3
Added decoding_tutorial\S1_R0_C4_Z17.tif. Stage: 1, Round: 0, Channel: 4
Added decoding_tutorial\S0_R1_C0_Z17.tif. Stage: 0, Round: 1, Channel: 0
Added decoding_tutorial\S0_R1_C1_Z17.tif. Stage: 0, Round: 1, Channel: 1
Added decoding_tutorial\S0_R1_C2_Z17.tif. Stage: 0, Round: 1, Channel: 2
Added decoding_tutorial\S0_R1_C3_Z17.tif. Stage: 0,

For verification, you can print out the size of the dataset.


In [2]:
num_stages, num_rounds, num_channels = issdata.get_dataset_shape()
print(f'There are {num_stages} number of stages')
print(f'There are {num_rounds} number of rounds')
print(f'There are {num_channels} number of channels')

There are 2 number of stages
There are 5 number of rounds
There are 5 number of channels


We can also verify that there are equal number of images for each stage, round and channel

In [3]:
issdata.is_dataset_complete()

True

(Optional) Let's take a look at the data using Napari.

In [2]:
import napari

# Select small piece of the data
small_data = issdata.select(stage=0, round=0)

# Load images into memory
small_data.load()

# Run Napari
viewer = napari.Viewer()
viewer.add_image(small_data.data.squeeze())
napari.run()

# Free memory
small_data.unload()

### Step 2. 2D Projection

In this step, we will perform a 2D projection of our data through maximum intensity projection. This involves selecting the maximum pixel value across different z-planes. To enhance the clarity of the 2D images, we can apply deconvolution. It's worth noting that deconvolution can be applied either before or after the 2D projection. However, it's important to highlight that deconvolution can be computationally intensive, often requiring a CUDA-supported GPU for efficient processing, especially when dealing with substantial stacks of 3D multiplexed images. For the purpose of this tutorial, we will omit the deconvolution step, but the necessary functions can be found in the `deconvolution.py` file.


In [4]:
# The iterate dataset allows us to iterate the dataset over stages, rounds and channels.
import numpy as np
from imaging_utils import imwrite
from os.path import join

for index, small_dataset in issdata.iterate_dataset(iter_stages=True, iter_rounds=True, iter_channels=True):
    # Load the small dataset
    small_dataset.load()
    # Get the image data
    data = small_dataset.data
    # MIP the data
    data = np.squeeze(data).max(axis=0)
    # Save the data
    imwrite(join('decoding_tutorial_2d','S{stage}_R{round}_C{channel}.tif'.format(**index)), data)
    # Finally, we unload the images (otherwise we might run oom)
    small_dataset.unload()

# Or equivalently ...
# from ISSDataset import mip
# mip(join('mip','stage{stage}_round{round}_channel{channel}.tif'), issdata)


### Step 3. Stitching

In this step, we will proceed to stitch the data using ASHLAR. This task can be accomplished by utilizing the `stitch_ashlar.py` function.


In [3]:
from os.path import join
# First we load the miped data
iss_data_miped = ISSDataContainer()
iss_data_miped.add_images_from_filepattern(join('decoding_tutorial_2d','S{stage}_R{round}_C{channel}.tif'))


Added decoding_tutorial_2d\S0_R0_C0.tif. Stage: 0, Round: 0, Channel: 0
Added decoding_tutorial_2d\S0_R0_C1.tif. Stage: 0, Round: 0, Channel: 1
Added decoding_tutorial_2d\S0_R0_C2.tif. Stage: 0, Round: 0, Channel: 2
Added decoding_tutorial_2d\S0_R0_C3.tif. Stage: 0, Round: 0, Channel: 3
Added decoding_tutorial_2d\S0_R0_C4.tif. Stage: 0, Round: 0, Channel: 4
Added decoding_tutorial_2d\S1_R0_C0.tif. Stage: 1, Round: 0, Channel: 0
Added decoding_tutorial_2d\S1_R0_C1.tif. Stage: 1, Round: 0, Channel: 1
Added decoding_tutorial_2d\S1_R0_C2.tif. Stage: 1, Round: 0, Channel: 2
Added decoding_tutorial_2d\S1_R0_C3.tif. Stage: 1, Round: 0, Channel: 3
Added decoding_tutorial_2d\S1_R0_C4.tif. Stage: 1, Round: 0, Channel: 4
Added decoding_tutorial_2d\S0_R1_C0.tif. Stage: 0, Round: 1, Channel: 0
Added decoding_tutorial_2d\S0_R1_C1.tif. Stage: 0, Round: 1, Channel: 1
Added decoding_tutorial_2d\S0_R1_C2.tif. Stage: 0, Round: 1, Channel: 2
Added decoding_tutorial_2d\S0_R1_C3.tif. Stage: 0, Round: 1, Cha

To successfully register and stitch the image data, it's crucial to have access to the initial position of each stage in pixel coordinates. This information can typically be extracted from the microscope software.

FAQ 1: If you get an error saying `Exception: Unable to find JAVA_HOME` you have to install OpenJDK11. On Windows, OpenJDK can be downloaded from [here](https://learn.microsoft.com/en-us/java/openjdk/download#openjdk-11). on Linux and macOS, see [this](https://openjdk.org/install/). Perhaps it can be installed through conda ...





In [1]:
from imaging_utils import stitch_ashlar, ISSDataContainer
from os.path import join
# First we load the miped data
iss_data_miped = ISSDataContainer()
iss_data_miped.add_images_from_filepattern(join('decoding_tutorial_2d','S{stage}_R{round}_C{channel}.tif'))

stage_locations = {
    0: (0, 0), 
    1: (0, 1843), 
}

# Stitch using ASHLAR
stitch_ashlar(join('decoding_tutorial_stitched','R{round}_C{channel}.tif'), iss_data_miped, stage_locations, reference_channel=4)

Added decoding_tutorial_2d\S0_R0_C0.tif. Stage: 0, Round: 0, Channel: 0
Added decoding_tutorial_2d\S0_R0_C1.tif. Stage: 0, Round: 0, Channel: 1
Added decoding_tutorial_2d\S0_R0_C2.tif. Stage: 0, Round: 0, Channel: 2
Added decoding_tutorial_2d\S0_R0_C3.tif. Stage: 0, Round: 0, Channel: 3
Added decoding_tutorial_2d\S0_R0_C4.tif. Stage: 0, Round: 0, Channel: 4
Added decoding_tutorial_2d\S1_R0_C0.tif. Stage: 1, Round: 0, Channel: 0
Added decoding_tutorial_2d\S1_R0_C1.tif. Stage: 1, Round: 0, Channel: 1
Added decoding_tutorial_2d\S1_R0_C2.tif. Stage: 1, Round: 0, Channel: 2
Added decoding_tutorial_2d\S1_R0_C3.tif. Stage: 1, Round: 0, Channel: 3
Added decoding_tutorial_2d\S1_R0_C4.tif. Stage: 1, Round: 0, Channel: 4
Added decoding_tutorial_2d\S0_R1_C0.tif. Stage: 0, Round: 1, Channel: 0
Added decoding_tutorial_2d\S0_R1_C1.tif. Stage: 0, Round: 1, Channel: 1
Added decoding_tutorial_2d\S0_R1_C2.tif. Stage: 0, Round: 1, Channel: 2
Added decoding_tutorial_2d\S0_R1_C3.tif. Stage: 0, Round: 1, Cha

### Step 4. Decoding

In this step, we will proceed to decode the previously stitched image data.

In [3]:
from imaging_utils import ISSDataContainer
# Load the stitched data
issdata = ISSDataContainer().add_images_from_filepattern(join('decoding_tutorial_stitched','R{round}_C{channel}.tif'))
# Load the data into memory
issdata.load()

Added decoding_tutorial_stitched\R0_C0.tif. Stage: 0, Round: 0, Channel: 0
Added decoding_tutorial_stitched\R0_C1.tif. Stage: 0, Round: 0, Channel: 1
Added decoding_tutorial_stitched\R0_C2.tif. Stage: 0, Round: 0, Channel: 2
Added decoding_tutorial_stitched\R0_C3.tif. Stage: 0, Round: 0, Channel: 3
Added decoding_tutorial_stitched\R0_C4.tif. Stage: 0, Round: 0, Channel: 4
Added decoding_tutorial_stitched\R1_C0.tif. Stage: 0, Round: 1, Channel: 0
Added decoding_tutorial_stitched\R1_C1.tif. Stage: 0, Round: 1, Channel: 1
Added decoding_tutorial_stitched\R1_C2.tif. Stage: 0, Round: 1, Channel: 2
Added decoding_tutorial_stitched\R1_C3.tif. Stage: 0, Round: 1, Channel: 3
Added decoding_tutorial_stitched\R1_C4.tif. Stage: 0, Round: 1, Channel: 4
Added decoding_tutorial_stitched\R2_C0.tif. Stage: 0, Round: 2, Channel: 0
Added decoding_tutorial_stitched\R2_C1.tif. Stage: 0, Round: 2, Channel: 1
Added decoding_tutorial_stitched\R2_C2.tif. Stage: 0, Round: 2, Channel: 2
Added decoding_tutorial_s

In [3]:
import pickle
import numpy as np
from os.path import join

# Load combinatorial labels (the codebook)
# The metadata file is available in the Google Drive
metadata = pickle.load(open(join('decoding_tutorial','metadata.pkl'), 'rb'))
codebook = metadata['codebook']
gene_names = list(codebook.keys())

# We need to create a 3D numpy array of shape (num_genes, num_rounds, num_channels)
# that containes the combinatorial labells in a one-hot format
n_codes, n_rounds, n_channels = len(codebook), 5, 5
codebook_numpy = np.zeros((n_codes, n_rounds, n_channels))
for gene_id, (gene, indices) in enumerate(codebook.items()):
    round_id, channel_id = indices['round_index'], indices['channel_index']
    codebook_numpy[gene_id, round_id, channel_id] = 1

print('Example code in the codebook:')
print(codebook_numpy[0])


Example code in the codebook:
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [9]:
from imaging_utils import ISSDataContainer
from decoding import istdeco_decode, estimate_fdr
import pandas as pd

issdata = ISSDataContainer().add_images_from_filepattern(join('decoding_tutorial_stitched','R{round}_C{channel}.tif'))

# Run the decoding
results = []
tile_idx = 1
for tile, origin in issdata.iterate_tiles(tile_height=512, tile_width=512, squeeze=True, use_vips=True):
    print(f'Decoding tile: {tile_idx}')
    tile_idx += 1
    # Decode the data using matrix factorization

    # Depending on your data, you might want to adjust the parameter min_integrated_intensity
    # or min_correct_spots
    # Usually a quality threshold between 0.5 and 0.85 works fine. 

    # This is really slow unless we can run on the GPU.
    decoded_table = istdeco_decode(tile, codebook_numpy, psf_sigma=(2.0, 2.0), device='cpu')

    decoded_table['Y'] = decoded_table['Y'] + origin[0]
    decoded_table['X'] = decoded_table['X'] + origin[1]
    results.append(pd.DataFrame(decoded_table))

    # Remove this to run over everything

results = pd.concat(results, axis=0)

Added decoding_tutorial_stitched\R0_C0.tif. Stage: 0, Round: 0, Channel: 0
Added decoding_tutorial_stitched\R0_C1.tif. Stage: 0, Round: 0, Channel: 1
Added decoding_tutorial_stitched\R0_C2.tif. Stage: 0, Round: 0, Channel: 2
Added decoding_tutorial_stitched\R0_C3.tif. Stage: 0, Round: 0, Channel: 3
Added decoding_tutorial_stitched\R0_C4.tif. Stage: 0, Round: 0, Channel: 4
Added decoding_tutorial_stitched\R1_C0.tif. Stage: 0, Round: 1, Channel: 0
Added decoding_tutorial_stitched\R1_C1.tif. Stage: 0, Round: 1, Channel: 1
Added decoding_tutorial_stitched\R1_C2.tif. Stage: 0, Round: 1, Channel: 2
Added decoding_tutorial_stitched\R1_C3.tif. Stage: 0, Round: 1, Channel: 3
Added decoding_tutorial_stitched\R1_C4.tif. Stage: 0, Round: 1, Channel: 4
Added decoding_tutorial_stitched\R2_C0.tif. Stage: 0, Round: 2, Channel: 0
Added decoding_tutorial_stitched\R2_C1.tif. Stage: 0, Round: 2, Channel: 1
Added decoding_tutorial_stitched\R2_C2.tif. Stage: 0, Round: 2, Channel: 2
Added decoding_tutorial_s

In [10]:
results

,Target id,Y,X,Intensity,Num explained spots,Rounds,Channels
0,1,404,112,23395.636719,3.675275,0;1;2;3;4,3;0;2;1;0
1,3,180,167,30096.474609,3.226670,0;1;2;3;4,1;2;2;2;3
2,5,511,107,9053.094727,3.300212,0;1;2;3;4,3;1;3;1;0
3,8,129,491,24767.859375,3.952892,0;1;2;3;4,0;1;0;1;0
4,8,248,157,7978.238281,3.298464,0;1;2;3;4,0;1;0;1;0
...,...,...,...,...,...,...,...
145,138,1598,3710,25788.039062,4.029922,0;1;2;3;4,0;3;3;2;2
146,138,1789,3605,47322.804688,4.169331,0;1;2;3;4,0;3;3;2;2
147,138,1807,3613,7578.980469,4.024141,0;1;2;3;4,0;3;3;2;2
148,143,1593,3681,12346.401367,3.550619,0;1;2;3;4,2;0;3;2;3


In [11]:
results['Gene' ] = [gene_names[id] for id in results['Target id']]
results


,Target id,Y,X,Intensity,Num explained spots,Rounds,Channels,Gene
0,1,404,112,23395.636719,3.675275,0;1;2;3;4,3;0;2;1;0,MLXIPL
1,3,180,167,30096.474609,3.226670,0;1;2;3;4,1;2;2;2;3,MXD1
2,5,511,107,9053.094727,3.300212,0;1;2;3;4,3;1;3;1;0,TIE1
3,8,129,491,24767.859375,3.952892,0;1;2;3;4,0;1;0;1;0,MNT
4,8,248,157,7978.238281,3.298464,0;1;2;3;4,0;1;0;1;0,MNT
...,...,...,...,...,...,...,...,...
145,138,1598,3710,25788.039062,4.029922,0;1;2;3;4,0;3;3;2;2,IGFBP7
146,138,1789,3605,47322.804688,4.169331,0;1;2;3;4,0;3;3;2;2,IGFBP7
147,138,1807,3613,7578.980469,4.024141,0;1;2;3;4,0;3;3;2;2,IGFBP7
148,143,1593,3681,12346.401367,3.550619,0;1;2;3;4,2;0;3;2;3,SMA/ACTA2


Some of the genes are marked as `Negatives` in the codebook. These genes correspond to non-biological labels that we do not expect to find in the data. Treating these negative genes as false-positives allow us to estimate a false discovery rate. This value is useful for quality control. 

In [12]:
positive_labels = [gene for gene in gene_names if 'Negative' not in gene]
negative_labels =  [gene for gene in gene_names if 'Negative' in gene]
fdr = estimate_fdr(results['Gene'], negative_labels, positive_labels)
print(f'False discovery rate is: {fdr}')

False discovery rate is: 0.019797979797943437


In [33]:
# We can also compute the quality for a different quality threshold 
fdr = estimate_fdr(results.query('`Num explained spots` > 3.6')['Gene'], negative_labels, positive_labels)
print(f'False discovery rate is: {fdr}')

False discovery rate is: 0.0


In [31]:
results.query('`Num explained spots` > 3.55')

,Target id,Y,X,Intensity,Num explained spots,Rounds,Channels,Gene
0,1,404,112,23395.636719,3.675275,0;1;2;3;4,3;0;2;1;0,MLXIPL
3,8,129,491,24767.859375,3.952892,0;1;2;3;4,0;1;0;1;0,MNT
6,16,461,208,34790.242188,3.563119,0;1;2;3;4,2;3;1;0;0,GGT5
7,28,216,505,19679.150391,3.860971,0;1;2;3;4,1;1;0;0;0,ENG
8,28,334,194,29034.261719,3.966337,0;1;2;3;4,1;1;0;0;0,ENG
...,...,...,...,...,...,...,...,...
144,136,1661,3687,28273.099609,4.105354,0;1;2;3;4,1;1;2;3;1,SPP1
145,138,1598,3710,25788.039062,4.029922,0;1;2;3;4,0;3;3;2;2,IGFBP7
146,138,1789,3605,47322.804688,4.169331,0;1;2;3;4,0;3;3;2;2,IGFBP7
147,138,1807,3613,7578.980469,4.024141,0;1;2;3;4,0;3;3;2;2,IGFBP7


An FDR < 1% is pretty OK